## test CNN pytorch

In [1]:
#Packages
! python3 -m pip install requests
! python3 -m pip install json
! python3 -m pip install matplotib
! python3 -m pip install time
! python3 -m pip install numpy 
! python3 -m pip install yfinance
! python3 -m pip install pandas
! python3 -m pip install torch


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement matplotib (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for matplotib
ERROR: Could not find a version that satisfies the requirement time (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for time

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release 

In [1]:
#Packages
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import yfinance as yf 
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import io
import time
import datetime

In [2]:
## Test CNN takes 256 x 256 image
class Torch_CNN(nn.Module):
    def __init__(self, embed, num_pred):
        super(Torch_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=embed, out_channels=8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(16, num_pred)  
    
    def forward(self, x): 
        x = torch.nn.functional.leaky_relu(self.conv1(x))  
        x = torch.nn.functional.leaky_relu(self.conv2(x))  
        x = self.gap(x)           
        x = x.view(x.size(0), -1)
        x = self.fc1(x)            
        return x

## Test FNN takes 4 ts values and CNN output
class Torch_FFN(nn.Module):
    def __init__(self, embed, hidden, num_pred):
        super(Torch_FFN, self).__init__()  
        self.fc1 = nn.Linear(embed, hidden) 
        self.fc2 = nn.Linear(hidden, hidden)  
        self.fc3 = nn.Linear(hidden, num_pred)  
    
    def forward(self, x):  
        x = torch.nn.functional.leaky_relu(self.fc1(x))
        x = torch.nn.functional.leaky_relu(self.fc2(x))
        x = self.fc3(x)            
        return x

# test Ens 
class Torch_Ens(nn.Module):
    def __init__(self, embed_img, embed_ts, hidden_dim, num_pred):
        super().__init__()
        self.cnn = Torch_CNN(embed_img, hidden_dim)  
        self.fnn = Torch_FFN(hidden_dim + embed_ts, hidden_dim, num_pred)
    
    def forward(self, x_img, y_ts):
        x = self.cnn(x_img)
        y_ts = y_ts.view(y_ts.size(0), -1)  
        combined = torch.cat([x, y_ts], dim=1)
        x = self.fnn(combined)
        return x

In [4]:
#Test
## El Model
model = Torch_Ens(
    embed_img=3, #Red / Green / Blue 
    embed_ts=4,      
    hidden_dim=256,  
    num_pred=1        
)

batch_size = 12
x_img = torch.randn(batch_size, 3, 256, 256)  
x_ts = torch.randn(batch_size, 4)             

print(f"Image shape: {x_img.shape}")
print(f"Time series shape: {x_ts.shape}")

output = model(x_img, x_ts)
#print(f"out shape: {output.shape}")  



Image shape: torch.Size([12, 3, 256, 256])
Time series shape: torch.Size([12, 4])


In [9]:
## crée un dataset on importe d'abord nos données
start_date =  "2020-02-01"
end_date = "2026-02-05"  
Tickers = ["ZC=F","ZW=F", "ZM=F", "GF=F", "LE=F"] #Corn / wheat / Soybean / Feeder cattle / live cattle
data_price = []
for ticker in Tickers:
    data = yf.download(ticker, start_date, progress = False, auto_adjust=False, interval="1d") #1year of data
    data_price.append(data['Adj Close'].pct_change().dropna())
data_price

df_close = pd.concat(data_price, axis=1)
df_close.columns = ["Corn","Wheat","Soy","Cow_food","Cows"] 

df_close


,Corn,Wheat,Soy,Cow_food,Cows
Date,,,,,
2020-02-04,0.009241,0.003150,-0.004486,0.006220,-0.000411
2020-02-05,-0.003924,0.008524,-0.003813,-0.013273,-0.007194
2020-02-06,-0.003940,-0.010231,0.002784,0.001658,0.003106
2020-02-07,0.011206,0.004494,0.003817,-0.005151,0.001651
2020-02-10,-0.004563,-0.012081,0.007259,0.003698,-0.008448
...,...,...,...,...,...
2026-01-30,-0.005804,-0.006464,-0.008108,-0.024108,0.001486
2026-02-02,-0.005838,-0.019052,0.003065,0.016862,0.009858
2026-02-03,0.006459,0.001895,-0.008829,0.004299,0.009027


In [10]:

latitude = 39.0997
longitude = -94.5786

url = "https://archive-api.open-meteo.com/v1/archive"
    
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date": end_date,
    "daily": "temperature_2m_mean",
    "timezone": "America/Chicago",
    "models": "best_match"
}
    

response = requests.get(url, params=params, timeout=30)

if response.status_code == 200:
    data = response.json()
    
    if 'daily' in data and 'time' in data['daily']:
        # Créer le DataFrame
        dates = pd.to_datetime(data['daily']['time'])
        temperatures = data['daily']['temperature_2m_mean']
        
        df_temp = pd.DataFrame({
            'date': dates,
            'temperature': temperatures
        })

df_temp = df_temp.set_index('date')
df_close = df_close.join(df_temp, how='inner')


In [16]:

params = {
"SERVICE": "WMS",
"REQUEST": "GetMap",
"VERSION": "1.3.0",
"LAYERS":  "MODIS_Terra_CorrectedReflectance_TrueColor", #Basic image
"CRS": "EPSG:4326",
"BBOX": "21,-128,49,-59",
"WIDTH": 256, # On baisse la résolution 
"HEIGHT": 256,
"FORMAT": "image/png",
"TIME": "01-01-2020", 
"TRANSPARENT": "FALSE"""
}

def Get_image_data(df):
    dates =  df.index.strftime('%Y-%m-%d').tolist() #On change le format


    images_dict={}
    for i, date in enumerate(dates):
        if i > 0:
            time.sleep(0.05)  #On est gentil avec l'API de la Nasa
        params['TIME'] = date 
        print(f"{params['TIME']}") 
        try:
            API_endpoint = "https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi"
            response = requests.get(API_endpoint, params=params, timeout=30)
            
            if response.status_code == 200:  # Success
                if response.content.startswith(b'\x89PNG'):  
                    img = mpimg.imread(io.BytesIO(response.content))
                    images_dict[date] = img
                else:
                    print(f"Response not PNG for {date}") 
                    print(response.text[:200]) 
                    
            else:
                print(f"Error {response.status_code} for {date}")  
    
        except Exception as e:  
            print(f"Erreur pour {date}: {str(e)}")  
    return images_dict

images_dict = Get_image_data(df_close)

2020-02-04


2020-02-05
2020-02-06
2020-02-07
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-20
2020-04-21
2020-04-22
2020-04-23
2020-04-24
2020-04-27
2020-04-28
2020-04-29
2020-04-30
2020-05-01
2020-05-04
2020-05-05
2020-05-06
2020-05-07
2020-05-08
2020-05-11
2020-05-12
2020-05-13
2020-05-14
2020-05-15
2020-05-18
2020-05-19
2020-05-20
2020-05-21
2020-05-22
2020-05-26
2020-05-27
2020-05-28
2020-05-29
2020-06-01
2020-06-02
2020-06-03
2020-06-04
2020-06-05
2020-06-08
2020-06-09
2020-06-10
2020-06-11
2020-06-12
2020-06-15

In [11]:
#np.savez_compressed('images.npz', **{date: img for date, img in images_dict.items()})
df_close.to_csv('prices.csv')

In [12]:
## On crée un dataset 
images_npz = np.load('images.npz', mmap_mode = 'r')
images_dict = {date: images_npz[date] for date in images_npz.files}
df_close = pd.read_csv('prices.csv', index_col=0, parse_dates=True)

del images_npz  # LA MEMOIRE

class full_dataset(Dataset):
    def __init__(self, images_dict, df, target_col='Corn'):
        self.images_dict = images_dict
        self.df = df
        self.target_col = target_col

        self.valid_indices = []
        dates = df.index.strftime('%Y-%m-%d').tolist()
        
        for i in range(len(dates) - 1):
            if dates[i] in images_dict:  # Current has image
                self.valid_indices.append(i)
        
        self.ts_mean = df.values.mean()
        self.ts_std = df.values.std()
        self.target_mean = df[target_col].mean()
        self.target_std = df[target_col].std()
    
    def __len__(self):
        return len(self.valid_indices) - 1 #la dernier val na pas de target
    
    def __getitem__(self, idx):
        df_idx = self.valid_indices[idx]
        date = self.df.index[df_idx].strftime('%Y-%m-%d')
        next_date = self.df.index[df_idx + 1]
        
        # Load image on-demand (not pre-loaded)
        img = self.images_dict[date]
        if img.ndim == 3 and img.shape[2] == 4:
            img = img[:, :, :3]
        img = img.transpose(2, 0, 1)
        
        # Get timeseries and target
        ts = self.df.iloc[df_idx].values
        target = self.df.loc[next_date, self.target_col]
        
        # Convert and normalize
        image = torch.from_numpy(img.copy()).float()
        ts = torch.from_numpy(ts.copy()).float()
        ts = (ts - self.ts_mean) / self.ts_std
        target = (target - self.target_mean) / self.target_std
        target = torch.tensor(target, dtype=torch.float32)
        
        return image, ts, target

dataset = full_dataset(images_dict,df_close)


In [ ]:
## On entraine 
model = Torch_Ens(
    embed_img=3, #Red / Green / Blue 
    embed_ts=6,      
    hidden_dim=128,  
    num_pred=1        
)

model.fnn.fc2.bias.data.fill_(0.0) # On initialise le bias a 0

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

split_idx = int(0.8 * len(dataset))  # 80 train, 20% test
train_dataset = torch.utils.data.Subset(dataset, range(split_idx))
val_dataset = torch.utils.data.Subset(dataset, range(split_idx, len(dataset)))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

model.train()
for epoch in range(20):

    model.train()
    total_loss = 0
    for image, ts, target in train_loader:
        image = image.float()
        ts = ts.float()
        target = target.float().unsqueeze(1)

        optimizer.zero_grad()
        output = model(image, ts)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()


    model.eval()
    val_loss = 0
    with torch.no_grad():
        for image, ts, target in val_loader:
            image = image.float()
            ts = ts.float()
            target = target.float().unsqueeze(1)
            output = model(image, ts)
            val_loss += criterion(output, target).item()

            predictions_v = []
            targets_v = []
            for images, time_series, targets in val_loader:
                predictions = model((images), time_series.float())
                predictions_v.append(predictions)
                targets_v.append(targets.unsqueeze(1))
            
            predictions_v = torch.cat(predictions_v, dim=0)
            targets_v = torch.cat(targets_v, dim=0)
            predictions_v = predictions_v.squeeze().numpy()
            targets_v = targets_v.squeeze().numpy()

            winrate = ((predictions_v > 0) == (targets_v > 0)).mean() * 100

            del predictions_v, targets_v

    print(f"Epoch {epoch}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Winrate : {winrate}%")


Epoch 0, Train Loss: 2.4121, Val Loss: 0.5593, Winrate : 50.331125827814574%
Epoch 1, Train Loss: 1.1340, Val Loss: 0.5314, Winrate : 50.331125827814574%


In [1]:
all_predictions = []
all_targets = []

with torch.no_grad():
    for images, time_series, targets in val_loader:
        predictions = model((images), time_series.float())
        all_predictions.append(predictions)
        all_targets.append(targets.unsqueeze(1))

all_predictions = torch.cat(all_predictions, dim=0)
all_targets = torch.cat(all_targets, dim=0)


plt.figure(figsize=(10,10))
plt.plot(all_targets, c='crimson')
plt.plot(all_predictions, c= 'blue')
plt.xlabel('Valeurs', fontsize=15)
plt.ylabel('Temps', fontsize=15)
plt.title("Prédites vs Réelles") 
plt.legend()
plt.show()


NameError: name 'torch' is not defined